In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_dir = '/content/drive/MyDrive/Clarifying_Questions_GPT_Research'

In [3]:
!cd {drive_dir} && pip install -r requirements_2.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.2 MB/s eta 0:00:00


In [4]:
import openai

import pickle

import json

import random

import re

import copy

import sys

import os

from tqdm import tqdm

import pprint


import numpy as np

import nltk

Code used for Analysis

In [5]:
def getChatQs(tree, node_id):
  context = tree['tree'][node_id]['context']
  chat_lst = [txt['content'] for txt in context]
  return [chat_lst[i] for i in range(len(chat_lst)) if i%2!=0][:-1]

def getIdsTxt(id_lst):
  return [tree['tree'][id]['generated_text'] for id in id_lst]

def getIdsQ(id_lst):
  sugg = tree['suggestion_ids']
  return [tree['tree'][id]['generated_text'] for id in id_lst if not id in sugg]

Loading Tree

In [6]:
with open(drive_dir+"/openai_key.key", "r") as f:
    openai.api_key = f.read()

In [7]:
def prepDic4Use(tree_dic):
  # turn lists into sets where we need a set instead of a list
  tree_dic['unfinished_ids'] = set(tree_dic['unfinished_ids'])
  tree_dic['suggestion_ids'] = set(tree_dic['suggestion_ids'])
  tree_dic['error_ids'] = set(tree_dic['error_ids'])
  return

def loadTreePkl(filename, pkl_path):
    # Opening pickle file
    with open(os.path.join(pkl_path, filename), 'rb') as f:
        # Reading from pickle file
        tree_dic = pickle.load(f)
    # turn lists into sets where we need a set instead of a list
    prepDic4Use(tree_dic)

    return tree_dic

In [8]:
# name of the pickle file that will be the saved tree
fname = 'GPTree_travel1_200nodes.pkl'
# path to folder that I want to save the tree in
fpath = '/content/drive/MyDrive/Clarifying_Questions_GPT_Research/Trees'

tree = loadTreePkl(fname, fpath)

General Stats

In [9]:
print(f"# of nodes: {len(tree['tree'])}\n")

layer_n = len(tree['ids_by_layer'])
print(f"max depth of tree: {layer_n}\n")
lns = '\n'.join([f"layer {l}: {len(tree['ids_by_layer'][l])}" for l in range(layer_n)])
print(f"nodes per layer:\n{lns}\n")

print(f"# of unfinished nodes: {len(tree['unfinished_ids'])}\n")
print(f"# of completly explored nodes: {len(tree['tree']) - len(tree['unfinished_ids'])}\n")

opt_count = 0
nodes_w_children_count = 0
for n in tree['tree']:
  branches = n['childrens_done_bool']
  if branches == None: continue
  real_child_n = len(n['generated_text'].split(' or '))
  nodes_w_children_count += 1
  opt_count += real_child_n
print(f"average number of options per node: {opt_count/nodes_w_children_count}\n")

print(f"# of completed chats: {len(tree['suggestion_ids'])}\n")

depth_count = 0
s_ids_set = tree['suggestion_ids']
for s_id in s_ids_set:
  #print(depth_count)
  depth_count += len(tree['tree'][s_id]['context']) / 2
print(f"average depth of a completed chat: {depth_count/len(s_ids_set)}\n")

print(f"# of error nodes: {len(tree['error_ids'])}")

# of nodes: 200

max depth of tree: 12

nodes per layer:
layer 0: 1
layer 1: 2
layer 2: 4
layer 3: 8
layer 4: 16
layer 5: 36
layer 6: 54
layer 7: 39
layer 8: 25
layer 9: 12
layer 10: 2
layer 11: 1

# of unfinished nodes: 91

# of completly explored nodes: 109

average number of options per node: 2.164285714285714

# of completed chats: 59

average depth of a completed chat: 8.220338983050848

# of error nodes: 1


In [10]:
id = 8
print((len(tree['tree'][id]['context']) - 2) / 2)
tree['tree'][id]['context']

8.0


[{'role': 'user',
  'content': 'Ask me one either or question at a time to help me find out where I want to go travel. Ask me many questions before giving me a suggestion'},
 {'role': 'assistant',
  'content': "Sure! Here's the first question: Would you prefer a beach destination or a city destination?"},
 {'role': 'user', 'content': 'city destination'},
 {'role': 'assistant',
  'content': 'Great! Next question: Would you like to visit a historic city or a modern metropolis?'},
 {'role': 'user', 'content': 'Historic city'},
 {'role': 'assistant',
  'content': 'Got it. Moving on to the next one: Are you more interested in ancient history or medieval history?'},
 {'role': 'user', 'content': 'ancient history'},
 {'role': 'assistant',
  'content': "Understood. Here's the next question: Would you prefer to explore a city known for its ancient ruins or one known for its well-preserved ancient architecture?"},
 {'role': 'user',
  'content': 'Explore a city known for its well-preserved ancient

Suggestion Stats

In [11]:
# name of the pickle file that will be the saved tree
json_name = fname[:-4]+'_id2suggestions.json'
# path to folder that I want to save the tree in
json_path = '/content/drive/MyDrive/Clarifying_Questions_GPT_Research/Trees/Analytics'

# loading suggestions dictionary
with open(os.path.join(json_path, json_name), 'r') as f:
  id2sugg = json.load(f)

In [12]:
id2sugg.values()

dict_values(['Prague, Czech Republic', 'Cancun, Mexico', ['Thailand', 'Phuket, Thailand', 'Koh Phi Phi, Thailand'], 'Phuket, Thailand', 'El Jem, Tunisia', 'Vancouver, Canada', 'Cornwall, England', 'Maldives', 'Pompeii, Italy', 'Maldives', ['Beaches of Southeast Asia', 'Bali, Indonesia', 'Phuket, Thailand', 'Langkawi, Malaysia'], 'Maldives', 'ERROR', 'Hoi An, Vietnam', 'Vancouver, Canada', 'Ibiza, Spain', 'Siem Reap, Cambodia', 'Kauai, Hawaii', 'Cancun, Mexico', 'Angkor, Cambodia', 'Prague, Czech Republic', 'Palomino Beach, Colombia', 'Bali, Indonesia', 'Rome, Italy', 'Cusco, Peru', 'Tokyo, Japan', 'Barcelona, Spain', 'Palawan, Philippines', 'Maldives', 'Barcelona, Spain', 'Athens, Greece', ['India', 'Angra, India', 'Karnataka, India'], 'Florida Keys, United States', 'El Nido, Philippines', 'Miami, Florida, USA', 'Epidaurus, Greece', 'Algarve region, Portugal', 'Raja Ampat, Indonesia', 'Zanzibar, Tanzania', 'Palawan, Philippines', 'Ghent, Belgium', 'Vienna, Austria', 'Cancun, Mexico', '

In [ ]:
from collections import Counter
countries = []
locations = []
for v in id2sugg.values():
  if type(v) == str:
    l = v.replace(' ', '')
    l = l.split(',')
    try:
      countries.append(l[1])
      locations.append(v)
    except:
      countries.append(v)
      locations.append(v)
  else:
    for vs in v[1:]:
      l = vs.replace(' ', '')
      l = l.split(',')
      try:
        countries.append(l[1])
        locations.append(vs)
      except:
        countries.append(vs)
        locations.append(vs)

countries = Counter(countries)
locations = Counter(locations)

print(len(countries))
pprint.pprint(countries)

31
Counter({'Spain': 7,
         'Thailand': 4,
         'Maldives': 4,
         'Indonesia': 4,
         'Philippines': 4,
         'Greece': 4,
         'Mexico': 3,
         'Colombia': 3,
         'CzechRepublic': 2,
         'Canada': 2,
         'Italy': 2,
         'Cambodia': 2,
         'Hawaii': 2,
         'India': 2,
         'Portugal': 2,
         'Tunisia': 1,
         'England': 1,
         'Malaysia': 1,
         'ERROR': 1,
         'Vietnam': 1,
         'Peru': 1,
         'Japan': 1,
         'UnitedStates': 1,
         'Florida': 1,
         'Tanzania': 1,
         'Belgium': 1,
         'Austria': 1,
         'Brazil': 1,
         'Turkey': 1,
         'Egypt': 1,
         'Dubai': 1})


In [ ]:
print(len(locations))
pprint.pprint(locations)

44
Counter({'Barcelona, Spain': 5,
         'Maldives': 4,
         'Cancun, Mexico': 3,
         'Phuket, Thailand': 3,
         'Bali, Indonesia': 3,
         'Prague, Czech Republic': 2,
         'Vancouver, Canada': 2,
         'Ibiza, Spain': 2,
         'Palawan, Philippines': 2,
         'Athens, Greece': 2,
         'Algarve region, Portugal': 2,
         'Koh Phi Phi, Thailand': 1,
         'El Jem, Tunisia': 1,
         'Cornwall, England': 1,
         'Pompeii, Italy': 1,
         'Langkawi, Malaysia': 1,
         'ERROR': 1,
         'Hoi An, Vietnam': 1,
         'Siem Reap, Cambodia': 1,
         'Kauai, Hawaii': 1,
         'Angkor, Cambodia': 1,
         'Palomino Beach, Colombia': 1,
         'Rome, Italy': 1,
         'Cusco, Peru': 1,
         'Tokyo, Japan': 1,
         'Angra, India': 1,
         'Karnataka, India': 1,
         'Florida Keys, United States': 1,
         'El Nido, Philippines': 1,
         'Miami, Florida, USA': 1,
         'Epidaurus, Greece': 1,
 

TFIDF Scores of Questions

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

def getScores_tfidf(corpus, stop_words = 'english'):
  vectorizer = TfidfVectorizer(stop_words=stop_words)
  tfidfs = vectorizer.fit_transform(corpus)

  words = [k for (v, k) in sorted((v, k)
            for k, v in vectorizer.vocabulary_.items())]
  scores = tfidfs.todense().tolist()

  return words, scores

def sortList1by2(list1, list2, reversed = True, include_zeros = True):
  zip_pairs = zip(list2, list1)

  if include_zeros:
    return [(x,y) for y, x in sorted(zip_pairs, reverse=reversed)]

  return [(x,y) for y, x in sorted(zip_pairs, reverse=reversed) if y > 0.0]


def getTopFeatures_tfidf(doc_lst, top_n=None, include_zeros = True, stop_words = 'english'):
  words, scores = getScores_tfidf(doc_lst, stop_words)
  if not top_n: top_n = len(words)

  return  [sortList1by2(words, scores[idx], True, include_zeros)[:top_n] for idx in range(len(doc_lst))]

In [14]:
def getAllQs(tree):
  return getIdsQ([id for id in range(len(tree['tree']))])

In [15]:
all_q = getAllQs(tree)

q_features = getTopFeatures_tfidf(all_q, None, False)
id = 0
print(all_q[id])
pprint.pprint(q_features[id])

Sure! Here's the first question: Would you prefer a beach destination or a city destination?
[('sure', 0.6735679088636594),
 ('destination', 0.5391136386249851),
 ('city', 0.29548382679178975),
 ('beach', 0.2899207472631523),
 ('prefer', 0.24347650541075733),
 ('question', 0.1581683170593485)]


Ordering Questions by Uniqueness (a sort of extra tfidf)

In [16]:
sorted([(4,3), (5,2), (4,3), (3,4)],key = lambda x: x[1])

[(5, 2), (4, 3), (4, 3), (3, 4)]

In [17]:
def scoreSumIDF(features_of_doc):
  return sum([y for x,y in features_of_doc]) / len(features_of_doc)

fs_ordered = [(idx, q_features[idx]) for idx in range(len(q_features))]

fs_ordered = sorted(fs_ordered, key = lambda x: scoreSumIDF(x[1]))

ids_by_fs = [fs[0] for fs in fs_ordered]

In [18]:
top_n = 10

print(f'Top {top_n} most unique questions')
# have to ignore the first one because it is a suggestion (mistake)
for id in ids_by_fs[1:top_n]:
  print(all_q[id])

print(f'\nBottom {top_n} least unique questions')
for id in ids_by_fs[len(ids_by_fs)-top_n:]:
  print(all_q[id])

Top 10 most unique questions
Noted! Here's the next question: Would you prefer a beach destination that offers a wide range of adventurous activities such as hiking, water sports, and zip-lining, or are you more interested in a destination that allows you to unwind and fully relax, with options for spa treatments and yoga retreats?
Sounds wonderful! Final question: Are you more inclined towards a city that is known for its hiking trails, camping spots, and outdoor sports, or a city that offers a mix of outdoor activities like skiing, snowboarding, and water sports depending on the season?
Noted! Here's the next question: Are you interested in a beach destination that offers a wide range of water sports and activities, such as jet skiing, parasailing, or kayaking, or do you prefer a more laid-back beach where you can simply relax and soak in the sun?
Great choice! Here's the next question: Are you more inclined towards a beach destination known for its vibrant local culture, traditional

TFIDF Scores of Questions per layer

In [19]:
def getLayerQs(tree):
  layer2qs = {}
  for l in range(len(tree['ids_by_layer'])):
    qs = getIdsQ(tree['ids_by_layer'][l])
    if len(qs) > 0:
      qs = '\n'.join(qs)
      layer2qs[l] = qs

  return layer2qs

In [20]:
stop_words = ['great', 'are', 'you', 'prefer', 'or', 'and', 'for', 'with', 'the', 'next', 'in', 'question',
              'to', 'do', 'its', 'interested', 'destination', 'would', 'is', 'that', 'more', 'known', 'where',
              'open', 'here', 'your', 'choice', 'noted', 'offers', 'perfect', 'one']
top_num = 20

layer2qs = getLayerQs(tree)
layer_qs = [layer2qs[k] for k in layer2qs.keys()]
layer_features = getTopFeatures_tfidf(layer_qs, top_num, stop_words=stop_words)

print(f'Top {top_num} tfidf scores of questions in each layer')
iter = 0
for fs in layer_features:
  print(f'layer #{iter}:\n{fs}\n')
  iter +=1

Top 20 tfidf scores of questions in each layer
layer #0:
[('first', 0.7262337852989004), ('sure', 0.5459220835724977), ('city', 0.3075629792234158), ('beach', 0.28276948486268816), ('zip', 0.0), ('yourself', 0.0), ('yoga', 0.0), ('year', 0.0), ('yacht', 0.0), ('world', 0.0), ('wonders', 0.0), ('wonderful', 0.0), ('without', 0.0), ('within', 0.0), ('willing', 0.0), ('wild', 0.0), ('widely', 0.0), ('wide', 0.0), ('white', 0.0), ('which', 0.0)]

layer #1:
[('trees', 0.31382004563441196), ('rugged', 0.31382004563441196), ('palm', 0.31382004563441196), ('metropolis', 0.31382004563441196), ('historic', 0.31382004563441196), ('cliffs', 0.31382004563441196), ('caves', 0.31382004563441196), ('tropical', 0.2682419504825221), ('beach', 0.24438062354013243), ('modern', 0.23590377733396523), ('secluded', 0.2108203415192698), ('visit', 0.19032568218207532), ('like', 0.1579875090335184), ('city', 0.132904073218823), ('zip', 0.0), ('yourself', 0.0), ('yoga', 0.0), ('year', 0.0), ('yacht', 0.0), ('worl

Doing Topic Modeling

In [21]:
with open(drive_dir+"/openai_key.key", "r") as f:
    openai.api_key = f.read()

In [22]:
# generates text using gpt-3.5 given a conversation context
def generateText(context):
  assert isinstance(context, list), f"input is not a context list, got {type(context)}"
  resp = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=context)

  return resp['choices'][0]['message']["content"]

# appends a new entry to a conversation context
def appendContext(text, context, role):
  assert role == "user" or role == "assistant", f"unexpected role, got: {role}"

  if role == "user":
    assert context[-1]["role"] == "assistant", f"incompatible adjecent role, got user"
    context.append({"role": "user", "content": text})
  else:
    assert context[-1]["role"] == "user", f"incompatible adjecent role, got assistant"
    context.append({"role": "assistant", "content": text})

  return

def getTopicPrompt(q):
  return f'Here is a question:\n{q}\n\nTask:\nSummarize the content of this either or question with 3 words; "this vs that".'

def getTopic(q):
  context = [{"role": "user", "content":getTopicPrompt(q)}]
  return generateText(context)

In [ ]:
q_topics = []
for q in tqdm(all_q):
  q_topics.append(getTopic(q))

100%|██████████| 141/141 [00:58<00:00,  2.40it/s]


In [ ]:
q_topics[:8]

['Beach vs city.',
 'Historic vs Modern.',
 'Ancient vs medieval.',
 'Ancient ruins vs architecture',
 'Greek Architecture vs Roman Architecture',
 'Amphitheaters vs temples.',
 'Ancient vs modern',
 'Italy vs Tunisia']

In [ ]:
topics_count = Counter()
for vsts in q_topics:
  vsts = vsts.lower()
  vsts = vsts.replace('.', '')
  ts = vsts.split('vs')
  for t in ts:
    if t[0] == ' ': t = t[1:]
    if t[-1] == ' ': t = t[:-1]
    if t in topics_count: topics_count[t] += 1
    else: topics_count[t] = 1

pprint.pprint(topics_count)

In [ ]:
topics_count = Counter()
for vsts in q_topics:
  vsts = vsts.lower()
  vsts = vsts.replace('.', '')
  ts = vsts.split(' ')
  for t in ts:
    if t[0] == ' ': t = t[1:]
    if t[-1] == ' ': t = t[:-1]
    if t in topics_count: topics_count[t] += 1
    else: topics_count[t] = 1

pprint.pprint(topics_count)

Analysing Redundancy

In [37]:
import itertools

def getRed(tree, node_id):

  qs = getChatQs(tree, node_id)

  qs = list(itertools.combinations(qs,2))

  ans = []

  for q1, q2, in tqdm(qs):
    # # getting the simplified questions
    # context = [{"role": "user", "content":f"Here is a question:\n1) {q1}\n\nTask:\nRewrite this question with the least number of words possible."}]
    # q1 = generateText(context)
    # appendContext(q1, context, "assistant")
    # appendContext(f"Here is another question:\n2) {q2}\n\nTask:\nRewrite this question with the least number of words possible.", context, "user")
    # q2 = generateText(context)

    # majority voting for wether questions overlap
    context = [{"role": "user", "content":f"Here are two questions:\n1) {q1}\n2) {q2}\n\nTask:\nAre these questions almost the same (Yes / No)?"}]
    # print(context)
    # print()
    votes = 0
    for i in range(10):
      if "yes" in generateText(context).lower(): votes += 1

    ans.append(votes)

  return qs, ans

In [39]:
qs, ans = getRed(tree, 34)
any([v > 5 for v in ans])

100%|██████████| 21/21 [01:02<00:00,  2.98s/it]


False

In [27]:
pairs = [qs[idx] for idx in range(len(ans)) if ans[idx] > 10]

In [28]:
pprint.pprint(pairs)

[("Understood. Here's the next question: Would you prefer to explore a city "
  'known for its ancient ruins or one known for its well-preserved ancient '
  'architecture?',
  "Great choice! Here's the next question: Would you prefer to visit a city "
  'known for its ancient Roman amphitheaters or one known for its ancient '
  'Roman temples and columns?'),
 ("Great choice! Here's the next question: Would you prefer to visit a city "
  'known for its ancient Roman amphitheaters or one known for its ancient '
  'Roman temples and columns?',
  "Sounds fantastic! Here's the next question: Would you prefer to visit a "
  'city in Italy that is home to the iconic Colosseum or one in Tunisia that '
  'houses the well-preserved Roman amphitheater of El Jem?'),
 ('Excellent! Moving on to the next question: Would you like to visit a city '
  'that is famous for its well-preserved ancient Roman amphitheater or one '
  'that is known for hosting modern events in a renovated ancient '
  'amphithe

In [36]:
pprint.pprint(pairs)

[("Understood. Here's the next question: Would you prefer to explore a city "
  'known for its ancient ruins or one known for its well-preserved ancient '
  'architecture?',
  "Great choice! Here's the next question: Would you prefer to visit a city "
  'known for its ancient Roman amphitheaters or one known for its ancient '
  'Roman temples and columns?'),
 ("Great choice! Here's the next question: Would you prefer to visit a city "
  'known for its ancient Roman amphitheaters or one known for its ancient '
  'Roman temples and columns?',
  "Sounds fantastic! Here's the next question: Would you prefer to visit a "
  'city in Italy that is home to the iconic Colosseum or one in Tunisia that '
  'houses the well-preserved Roman amphitheater of El Jem?'),
 ('Excellent! Moving on to the next question: Would you like to visit a city '
  'that is famous for its well-preserved ancient Roman amphitheater or one '
  'that is known for hosting modern events in a renovated ancient '
  'amphithe